In [37]:
import json
from io import open

def preprocess_data(file_path):
    train_data = {}

    with open(file_path, 'r') as file:
        i = 0
        for line in file:
            # Parse the line as JSON
            train_data[i] = json.loads(line)
            i+=1

    inputs, targets = [], []

    for key, instance in train_data.items():
        if isinstance(instance, dict) and 'messages' in instance:
            messages = instance['messages']
            game_scores = instance['game_score']

        for idx, (message, score) in enumerate(zip(messages, game_scores)):
            start_index = max(0, idx-5)
            end_index = min(len(messages)-1, idx)
            context = " ".join(messages[start_index:end_index])
            inputs.append(f"Given the following context, generate an appropriate repsonse: {context}")
            targets.append(message)
    return inputs, targets

training_inputs, training_targets = preprocess_data("2020_acl_diplomacy/data/train.jsonl")
validation_inputs, validation_targets = preprocess_data("2020_acl_diplomacy/data/validation.jsonl")
test_inputs, test_targets = preprocess_data("2020_acl_diplomacy/data/test.jsonl")

print(len(training_inputs)) 
print(len(training_targets))
print(training_inputs[3])
print(training_targets[3])

13132
13132
Given the following context, generate an appropriate repsonse: Germany!

Just the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. 

So...do I suggest it?

I’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.

What is your appetite like for unusual and crazy? You've whet my appetite, Italy. What's the suggestion? 👍
It seems like there are a lot of ways that could go wrong...I don't see why France would see you approaching/taking Munich--while I do nothing about it--and not immediately feel skittish


In [38]:
%pip install transformers
%pip install torch

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

class DiplomacyDataset(Dataset):
    def __init__(self, data, targets, tokenizer, max_len=512):
        self.data = data
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        tokenized_input = self.tokenizer.encode(self.data[index], max_length=self.max_len, truncation=True, padding='max_length', return_tensors='pt')
        tokenized_target = self.tokenizer.encode(self.targets[index], max_length=self.max_len, truncation=True, padding='max_length', return_tensors='pt')
        item = {
            'input_ids': tokenized_input.flatten(),
            # 'decoder_input_ids': tokenized_target.flatten()[:-1], # remove the last token of the target
            'labels': tokenized_target.flatten()[1:],
        }
        return item

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

train_dataset = DiplomacyDataset(training_inputs, training_targets, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

validation_dataset = DiplomacyDataset(validation_inputs, validation_targets, tokenizer)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=True)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
%pip install accelerate -U
import accelerate


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-4)


# for epoch in range(3):
#     model.train()
#     for batch in train_loader:
#         input_ids = batch['input_ids'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, labels=labels)
#         loss = outputs[0]

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         print(f"Epoch: {epoch}, Loss: {loss.item()}")
        


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    num_train_epochs=3,
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
trainer.train() 

  0%|          | 10/9849 [00:20<3:22:21,  1.23s/it]

{'loss': 2.3186, 'learning_rate': 5.999999999999999e-06, 'epoch': 0.0}


  0%|          | 20/9849 [00:31<3:02:11,  1.11s/it]

{'loss': 2.1954, 'learning_rate': 1.1999999999999999e-05, 'epoch': 0.01}


  0%|          | 30/9849 [00:43<3:07:30,  1.15s/it]

...

{'loss': 0.203, 'learning_rate': 0.0002881270724141619, 'epoch': 0.27}


  9%|▉         | 880/9849 [16:33<2:52:51,  1.16s/it]

{'loss': 0.1868, 'learning_rate': 0.00028780618247940956, 'epoch': 0.27}


  9%|▉         | 885/9849 [16:38<2:46:58,  1.12s/it]

KeyboardInterrupt: 

In [ ]:
test_dataset = DiplomacyDataset(test_inputs, test_targets, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

model.eval()
input_ids = test_dataset[10]['input_ids'].unsqueeze(0).to(device)
outputs = model.generate(input_ids)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
